In [2]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

In [3]:
with open('./data/uniprot-compressed_true_download_true_format_json_query__28_28taxono-2023.02.05-17.45.13.08.json', 'r') as f:
    data = json.load(f)

In [4]:
records = []
results = data['results']
for result in results:
    primary_accession = result['primaryAccession']
    uniprotkb_id = result['uniProtkbId']
    if 'sequence' in result:
        sequence = result['sequence']['value']
        if 'genes' in result and len(result['genes']) != 0:
            for gene in result['genes']:
                gene_names = []
                if 'geneName' in gene:
                    gene_names.append(gene['geneName'])
                if 'synonyms' in gene:
                    gene_names.extend(gene['synonyms'])
                if 'orfNames' in gene:
                    gene_names.extend(gene['orfNames'])
                for gene_name in gene_names:
                    if 'value' in gene_name:
                        gene_symbol = gene_name['value']
                        records.append({
                            'gene_symbol': gene_symbol,
                            'primary_accession': primary_accession,
                            'uniprotkb_id': uniprotkb_id,
                            'sequence': sequence
                        })

In [12]:
df = pd.DataFrame(records)

df['gene_symbol_harmonized'] = df['gene_symbol'].str.strip().str.lower()

df_gene_symbol_protein_sequence = df.copy()

In [16]:
df = pd.read_pickle('./data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023.pkl')

df['gene_symbol_harmonized'] = df['gene_symbol'].str.strip().str.lower()

df = df.merge(df_gene_symbol_protein_sequence, how='left', on='gene_symbol_harmonized')

df = df.drop(['gene_symbol_harmonized'], axis=1)

df.to_pickle('./data/effect_size_protein_sequence_source.pkl')